In [2]:
import pandas as pd
from src.utils import load_data, load_config

In [15]:
df = pd.read_csv(r"C:\Users\voliveira\OneDrive - Corporacion Empresarial Altra SL\00-Proyectos\myzone\DATA\processed\2024-05-14\text_to_analyse_clean.csv",
                 sep='¬', encoding='utf-8', engine='python')

In [16]:
df.shape

(25661, 19)

In [17]:
df.sample(10)

,codigo,id_pieza,desc_problema_translated,descripcion_translated,problema_translated,cod_articulo,text_to_analyse,CODART_A3,Fuzzy_Score,CODART,DESCART,CAR1,CAR2,CAR3,CAR4,DESCCAR1,DESCCAR2,DESCCAR3,DESCCAR4
7513,AGZUZGTT62,22818.0,"ENTRADA 36371\r\n""ANALIZAR I+D""",TERMOSTATO,"ENTRADA 36371\r\n\""ANALIZAR I+D\""",AZCE6BLUEFACECB,"ENTRADA 36371\r\n""ANALIZAR I+D"" TERMOSTATO ENT...",AZCE6BLUEFACECB,100.0,AZCE6BLUEFACECB,Termostato cable a color Airzone Blueface blan...,1.0,250.0,91.0,1.0,SISTEMAS DE ZONAS,FLEXA (CE6),TERMOSTATOS,BLUEFACE
5443,NGMVM5ZP60,15751.0,EL TECLADO NO GUNCIONA CORRECTAMENTE. NUESTRO ...,TERMOSTATO.DIST.INTELLIG.BLUEFACE 32 Z.BL.,FRONTAL DESPEGADO.,AZDI6BLUEFACECB,EL TECLADO NO GUNCIONA CORRECTAMENTE. NUESTRO ...,AZDI6BLUEFACECB,100.0,AZDI6BLUEFACECB,Termostato cable a color Airzone Blueface blan...,1.0,251.0,91.0,1.0,SISTEMAS DE ZONAS,ACUAZONE (DI6),TERMOSTATOS,BLUEFACE
8017,BMDTYMTQD0,24798.0,LA PANTALLA NO HACE NADA.,OTT. CARA AZUL CB,LA PANTALLA NO HACE NADA.,AZCE6BLUEFACECB,LA PANTALLA NO HACE NADA. OTT. CARA AZUL CB LA...,AZCE6BLUEFACECB,100.0,AZCE6BLUEFACECB,Termostato cable a color Airzone Blueface blan...,1.0,250.0,91.0,1.0,SISTEMAS DE ZONAS,FLEXA (CE6),TERMOSTATOS,BLUEFACE
9698,MMZVNG1Q68,31088.0,EXPEDIENTE 3228,TERMOSTATO IBPRO6 INTELIGENTE AIRZONE FILTRO B...,EXPEDIENTE 3228,AZCE6BLUEFACECB,EXPEDIENTE 3228 TERMOSTATO IBPRO6 INTELIGENTE ...,AZCE6BLUEFACECB,100.0,AZCE6BLUEFACECB,Termostato cable a color Airzone Blueface blan...,1.0,250.0,91.0,1.0,SISTEMAS DE ZONAS,FLEXA (CE6),TERMOSTATOS,BLUEFACE
2201,Y2MABJTP89,6731.0,Ver con Eddy para actualizaciones de instalación,SERVIDOR WEB,ACTUALIZACIÓN DE INSTALACIÓN,AZX6WEBSCLOUDR,Ver con Eddy para actualizaciones de instalaci...,AZX6WEBSCLOUDR,100.0,AZX6WEBSCLOUDR,Webserver Airzone cloud Wi-Fi,1.0,260.0,93.0,NaN,SISTEMAS DE ZONAS,COMUNES,WEB SERVER & INTEGRACION,NaN
6937,ZMPTZWNPB6,21006.0,regulador HS,Regulador 0-10V (canalizable 304),regulador HS,AZX6010VOLTSZ,regulador HS Regulador 0-10V (canalizable 304)...,AZX6010VOLTSZ,100.0,AZX6010VOLTSZ,Pasarela control Airzone-fancoil 0-10V,1.0,260.0,49.0,NaN,SISTEMAS DE ZONAS,COMUNES,PASARELAS,NaN
12775,AW1TZWLS71,40564.0,REJILLA SE QUEDA SIN FUERZA AL ABRIR Y NO TERM...,REJILLA INTELIGENTE DOBLE AIRZONE 800X100 MOTO...,REJILLA SIN FUERZA AL ABRIR Y SE QUEDA A MEDIAS,RINT080010BKMTE,REJILLA SE QUEDA SIN FUERZA AL ABRIR Y NO TERM...,RINT080010BKMTE,100.0,RINT080010BKMTE,Rejilla Inteligente doble Airzone motorizada 8...,1.0,264.0,31.0,92.0,SISTEMAS DE ZONAS,DIFUSION MOTORIZADA,REJILLAS,RINT
12155,AM5QZWDX2D,38702.0,EL WEBSERVER 5G ESTA EN ERROR 12 Y HA PERDIDO ...,SERVIDOR WEB AIRZONE NUBE WIFI 5G,1 UNIDAD.,AZX6WSC5GER,EL WEBSERVER 5G ESTA EN ERROR 12 Y HA PERDIDO ...,AZX6WSC5GER,100.0,AZX6WSC5GER,Webserver Airzone cloud Wi-Fi dual 2.4-5G,1.0,260.0,93.0,NaN,SISTEMAS DE ZONAS,COMUNES,WEB SERVER & INTEGRACION,NaN
11803,MGHPLGTSEF,37628.0,Mando no responde,ENVIO AIRZONE,Mando no funciona.,ZONACOXTBR,Mando no responde ENVIO AIRZONE Mando no funci...,ZONACOXTBR,100.0,ZONACOXTBR,TERMOSTATO ZONA TACTO CORDLESS BLANCO,1.0,243.0,91.0,109.0,SISTEMAS DE ZONAS,UNIVERSAL,TERMOSTATOS,TERMOSTATOS
10465,L2XQMM5UF4,33532.0,REJILLAS HAN LLEGADO GOLPEADAS POR EL TRANSPORTE,NaN,GOLPEADA,RSDR020015BKX,REJILLAS HAN LLEGADO GOLPEADAS POR EL TRANSPOR...,RSDR020015BKX,100.0,RSDR020015BKX,Rejilla simple deflex h Airzone 200x150 mm bla...,2.0,262.0,31.0,81.0,DIFUSIÓN,DIFUSION NO MOTORIZADA,REJILLAS,RSDR


In [35]:
config = load_config(
    file_name='processing_config', 
    env='prod',
    folder='../../config'
)

In [36]:
from src.preprocessing.utils import pre_process_text_spacy

test_str = [
    "El producto no funciona? 1341 Alo alo "
    "alo. AZCE6BLUECB",
    "TTO. TACTO: NO RESPONDE CORRECTAMENTE EL TACTIL TTO. TACTO CABLE SUPERF. BL. TTO. TACTO: NO RESPONDE CORRECTAMENTE EL TACTIL",
]

pre_process_text_spacy(test_str, **config.processing['tokenizer'])

['el producto no funciona 1341 alo alo alo azce6bluecb',
 'tto tacto no responde correctamente el tactil tto tacto cable superf bl tto tacto no responde correctamente el tactil']